## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**1241. Number of Comments per Post(Easy)**

**Table: Submissions**

| Column Name   | Type     |
|---------------|----------|
| sub_id        | int      |
| parent_id     | int      |

This table may have duplicate rows.
Each row can be a post or comment on the post.
parent_id is null for posts.
parent_id for comments is sub_id for another post in the table.
 
**Write a solution to find the number of comments per post. The result table should contain post_id and its corresponding number_of_comments.**

The Submissions table may contain duplicate comments. You should count the number of unique comments per post.

The Submissions table may contain duplicate posts. You should treat them as one post.

The result table should be ordered by post_id in ascending order.

The result format is in the following example.

**Example 1:**

**Input:** 

**Submissions table:**

| sub_id  | parent_id  |
|---------|------------|
| 1       | Null       |
| 2       | Null       |
| 1       | Null       |
| 12      | Null       |
| 3       | 1          |
| 5       | 2          |
| 3       | 1          |
| 4       | 1          |
| 9       | 1          |
| 10      | 2          |
| 6       | 7          |

**Output:** 
| post_id | number_of_comments |
|---------|--------------------|
| 1       | 3                  |
| 2       | 2                  |
| 12      | 0                  |

**Explanation:** 
- The post with id 1 has three comments in the table with id 3, 4, and 9. The comment with id 3 is repeated in the table, we counted it only once.
- The post with id 2 has two comments in the table with id 5 and 10.
- The post with id 12 has no comments in the table.
- The comment with id 6 is a comment on a deleted post with id 7 so we ignored it.

In [0]:
submissions_data_1241 = [
    (1, None), (2, None), (1, None), (12, None),
    (3, 1), (5, 2), (3, 1), (4, 1), (9, 1),
    (10, 2), (6, 7)
]

submissions_columns_1241 = ["sub_id", "parent_id"]
submissions_df_1241 = spark.createDataFrame(submissions_data_1241, submissions_columns_1241)
submissions_df_1241.show()

+------+---------+
|sub_id|parent_id|
+------+---------+
|     1|     NULL|
|     2|     NULL|
|     1|     NULL|
|    12|     NULL|
|     3|        1|
|     5|        2|
|     3|        1|
|     4|        1|
|     9|        1|
|    10|        2|
|     6|        7|
+------+---------+



In [0]:
posts_df_1241 = submissions_df_1241\
                    .filter(col("parent_id").isNull())\
                        .select(col("sub_id").alias("post_id")).dropDuplicates()

comments_df_1241 = submissions_df_1241\
                        .filter(col("parent_id").isNotNull())\
                            .select("sub_id", "parent_id").dropDuplicates()

In [0]:
comments_on_posts_1241 = comments_df_1241\
                                .join(posts_df_1241,comments_df_1241.parent_id == posts_df_1241.post_id, how="inner")

comment_counts_1241 = comments_on_posts_1241\
                                .groupBy("post_id").agg(countDistinct("sub_id").alias("number_of_comments"))

In [0]:
posts_df_1241\
    .join(comment_counts_1241, on="post_id", how="left").fillna(0).orderBy("post_id").show()

+-------+------------------+
|post_id|number_of_comments|
+-------+------------------+
|      1|                 3|
|      2|                 2|
|     12|                 0|
+-------+------------------+

